
<div dir = "rtl" style = 'font-family: "B Zar";' align = "center">
    <h1 align = "center">
        به نام خدا
    </h1>
    <h3 align = "center">
        تمرین چهارم پردازش زبان - تشخیص موجودیت های نامدار مدل برت
    </h3>
    <h3 align = "center">
        استاد: دکتر عسگری
    </h3>
    <h3 align = "center">
         محمدرضا کمالی -      
         مهدی سعیدی  
    </h3>

</div>


# Libraries and Tools


<div dir = "rtl" style = 'font-family: "B Zar";'>

  <p>
کتابخانه ها و ابزار های مورد نیاز را نصب و اضافه می کنیم
 </p>
</div>


In [ ]:
%pip install transformers
%pip install keras
%pip install scikit_learn
%pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%pip install keras_preprocessing

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import transformers
from transformers import BertForTokenClassification, AdamW
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig, BertModel
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import get_linear_schedule_with_warmup
import joblib
from seqeval.metrics import f1_score,recall_score,precision_score, accuracy_score

# Load & Pre Process Data


<div dir = "rtl" style = 'font-family: "B Zar";'>
  <p>در این بخش ابتدا ما از داده های خام نظرات فیلم دیتا فریم میسازیم و سعی در پیش پردازش داده ها کردیم سپس بعد از ساختن دیتا فریم بخش مربوطه را کامنت کردیم و فایل های csv نهایی را به پیوست اورده ایم. </p>

</div>


In [ ]:
# uncommet for create dataframe from raw data
# columns = ["sentence_id" , "word" , "tag"]
# data = pd.DataFrame([], columns=columns)

# file1 = open('/content/engtest.bio', 'r')
# Lines = file1.readlines()
  
# count = 1
# for line in Lines:

#     if(len((line.strip()).split('\t')) == 1):
#       count += 1
#       print()
#     else:
#       data = data.append({'sentence_id':count,'word':(line.strip()).split('\t')[1],'tag':(line.strip()).split('\t')[0]}, ignore_index = True)
# file1.close()
# file2 = open('/content/trivia10k13test.bio', 'r')
# Lines = file2.readlines()
  
# for line in Lines:

#     if(len((line.strip()).split('\t')) == 1):
#       count += 1
#     else:
#       data = data.append({'sentence_id':count,'word':(line.strip()).split('\t')[1],'tag':(line.strip()).split('\t')[0]}, ignore_index = True)
# file2.close()


In [ ]:
# uncommet for create dataframe from raw data

# data

,sentence_id,word,tag
0,1,are,O
1,1,there,O
2,1,any,O
3,1,good,O
4,1,romantic,B-GENRE
...,...,...,...
63716,4396,the,I-Origin
63717,4396,novel,I-Origin
63718,4396,by,I-Origin
63719,4396,ben,I-Origin


In [ ]:
# uncommet for create dataframe from raw data

# pd.DataFrame(data).to_csv("/content/imdb_test.csv")

In [ ]:
# uncommet for create dataframe from raw data

# data = pd.read_csv("/content/imdb_test.csv")
# for i in range(len(data['tag'])):
#   data['tag'].iloc[i] = data['tag'].iloc[i].upper()
# pd.DataFrame(data).to_csv("/content/imdb_test2.csv")


<div dir = "rtl" style = 'font-family: "B Zar";'>
  <p>سپس از روی دیتا فریم سعی کردیم زوج لیست ، جملات و تگ هر کلمعه را پیدا کنیم سپس کلمات جملات را با توکنایزر برت انکود کردیم و سپس به طولی واحد برای ورودی دادن به ترنسفرمر برده ایم. سپس با ایجاد 40 کلاس از تمامی کلاس هایی که این دیتا ست برای موجودیت های نامدار داشت سعی کرده ایم تگ بزنیم به هر کلمه سپس یک ماسک اتنشن نیز تعریف کرده ایم تا کلمات حاشیه را مدل برت متوجه نشود و با تگ PAD مشخص شده اند. </p>
</div>


In [ ]:
data = pd.read_csv("/content/imdb_training2.csv", encoding="latin1", skip_blank_lines=False).fillna(method="ffill")

In [ ]:
class Dataset(object):

    def __init__(self, data):
        self.counter = 1
        self.data = data
        agg_func = lambda s: [(word, tag) for word, tag in zip(s["word"].values.tolist(),s["tag"].values.tolist())]
        self.grouped = self.data.groupby("sentence_id").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.counter)]
            self.counter += 1
            return s
        except:
            return None

In [ ]:
dataset = Dataset(data)
sentences = [[word[0] for word in sentence] for sentence in dataset.sentences]
print(sentences[0])

['what', 'movies', 'star', 'bruce', 'willis']


In [ ]:
tags = [[s[1] for s in sentence] for sentence in dataset.sentences]
print(tags[0])

['O', 'O', 'O', 'B-ACTOR', 'I-ACTOR']


In [ ]:
tag_values = list(set(data["tag"].values))
tag_values.append("PAD")
tag_values.sort()
print(tag_values)

['B-ACTOR', 'B-AWARD', 'B-CHARACTER', 'B-CHARACTER_NAME', 'B-DIRECTOR', 'B-GENRE', 'B-OPINION', 'B-ORIGIN', 'B-PLOT', 'B-QUOTE', 'B-RATING', 'B-RATINGS_AVERAGE', 'B-RELATIONSHIP', 'B-REVIEW', 'B-SONG', 'B-SOUNDTRACK', 'B-TITLE', 'B-TRAILER', 'B-YEAR', 'I-ACTOR', 'I-AWARD', 'I-CHARACTER', 'I-CHARACTER_NAME', 'I-DIRECTOR', 'I-GENRE', 'I-OPINION', 'I-ORIGIN', 'I-PLOT', 'I-QUOTE', 'I-RATING', 'I-RATINGS_AVERAGE', 'I-RELATIONSHIP', 'I-REVIEW', 'I-SONG', 'I-SOUNDTRACK', 'I-TITLE', 'I-TRAILER', 'I-YEAR', 'O', 'PAD']


In [ ]:
tag2idx = {t: i for i, t in enumerate(tag_values)}
print(tag2idx)

{'B-ACTOR': 0, 'B-AWARD': 1, 'B-CHARACTER': 2, 'B-CHARACTER_NAME': 3, 'B-DIRECTOR': 4, 'B-GENRE': 5, 'B-OPINION': 6, 'B-ORIGIN': 7, 'B-PLOT': 8, 'B-QUOTE': 9, 'B-RATING': 10, 'B-RATINGS_AVERAGE': 11, 'B-RELATIONSHIP': 12, 'B-REVIEW': 13, 'B-SONG': 14, 'B-SOUNDTRACK': 15, 'B-TITLE': 16, 'B-TRAILER': 17, 'B-YEAR': 18, 'I-ACTOR': 19, 'I-AWARD': 20, 'I-CHARACTER': 21, 'I-CHARACTER_NAME': 22, 'I-DIRECTOR': 23, 'I-GENRE': 24, 'I-OPINION': 25, 'I-ORIGIN': 26, 'I-PLOT': 27, 'I-QUOTE': 28, 'I-RATING': 29, 'I-RATINGS_AVERAGE': 30, 'I-RELATIONSHIP': 31, 'I-REVIEW': 32, 'I-SONG': 33, 'I-SOUNDTRACK': 34, 'I-TITLE': 35, 'I-TRAILER': 36, 'I-YEAR': 37, 'O': 38, 'PAD': 39}


In [ ]:
LEN = 75
batch_size = 200
epoch = 4
max_grad_norm = 1.0
device = torch.device("cuda")
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=False)

In [ ]:
def tokenizing_sentence(sentence, tag):
    tokenized_sentence = []
    tags = []

    for word, tag in zip(sentence, tag):
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)
        tokenized_sentence.extend(tokenized_word)
        tags.extend([tag] * n_subwords)

    return tokenized_sentence, tags

In [ ]:
sentencse_tags = [
    tokenizing_sentence(sent, tags)
    for sent, tags in zip(sentences, tags)
]

In [ ]:
sentencse_tags[0]

(['what', 'movies', 'star', 'br', '##uce', 'will', '##is'],
 ['O', 'O', 'O', 'B-ACTOR', 'B-ACTOR', 'I-ACTOR', 'I-ACTOR'])

In [ ]:
len(sentencse_tags)

17591

In [ ]:
all_texts = [token_tag_pair[0] for token_tag_pair in sentencse_tags]

In [ ]:
all_texts[0]

['what', 'movies', 'star', 'br', '##uce', 'will', '##is']

In [ ]:
tags = [token_tag_pair[1] for token_tag_pair in sentencse_tags]


In [ ]:
tags[0]

['O', 'O', 'O', 'B-ACTOR', 'B-ACTOR', 'I-ACTOR', 'I-ACTOR']

In [ ]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in all_texts],maxlen=LEN, dtype="long", value=0.0,truncating="post", padding="post")

In [ ]:
input_ids[0]

array([ 1184,  5558,  2851,  9304, 15776,  1209,  1548,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0])

In [ ]:
tags = pad_sequences([[tag2idx.get(l) for l in tagi] for tagi in tags],maxlen=LEN, value=tag2idx["PAD"], padding="post",dtype="long", truncating="post")

In [ ]:
tags[0]

array([38, 38, 38,  0,  0, 19, 19, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
       39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
       39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
       39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39, 39,
       39, 39, 39, 39, 39, 39, 39])

In [ ]:
attention_masks = [[float(k != 0.0) for k in i] for i in input_ids]

In [ ]:
attention_masks[0]

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [ ]:
train_inputs, val_inputs, train_tags, val_tags = train_test_split(input_ids, tags,test_size=0.2)
train_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,test_size=0.2)

In [ ]:
train_inputs = torch.tensor(train_inputs)
val_inputs = torch.tensor(val_inputs)
train_tags = torch.tensor(train_tags)
val_tags = torch.tensor(val_tags)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)


In [ ]:
train1 = TensorDataset(train_inputs, train_masks, train_tags)
train2 = RandomSampler(train1)
train_batches = DataLoader(train1, sampler=train2, batch_size=batch_size)
valid1 = TensorDataset(val_inputs, val_masks, val_tags)
valid2 = SequentialSampler(valid1)
eval_batches = DataLoader(valid1, sampler=valid2, batch_size=batch_size)


<div dir = "rtl" style = 'font-family: "B Zar";'>
  <p>و در نهایت هر 3 این داده ها در قالب یه داده به مدل برت به عنوان ورودی داده میشود.</p>

</div>


In [ ]:
train1[0]

(tensor([ 1184,  1108,  1115, 22572, 11657,  1158, 27629,  8928,  2523,  1187,
          1119,  1307,   170,  5176,  1133,  1136,  1126,  2168,  2523,   170,
          6376,  2523,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0]),
 tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0.]),
 tensor([38, 38, 38,  0,  0,  0, 19, 19, 38, 38, 38, 3

# Bert Model


<div dir = "rtl" style = 'font-family: "B Zar";'>
  <p>در این مرحله مدل برت بر روی دادگان ما به تفکیک 80 به 20 داده اموزش و ارزیابی تقسیم میشود و اموزش میبیبند. و در هر مرحله 4 معیار گفته شده را برای داده ها گزارش میکینم </p>

</div>


In [ ]:
model = BertForTokenClassification.from_pretrained("bert-base-cased",num_labels=len(tag2idx),output_attentions = False,output_hidden_states = False)
model.cuda();
param_optimizer = list(model.classifier.named_parameters())
optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = AdamW(optimizer_grouped_parameters,lr=1e-1,eps=1e-8)
steps = len(train_batches) * epoch
scheduler = get_linear_schedule_with_warmup(optimizer,num_warmup_steps=0,num_training_steps=steps)


Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

In [ ]:
loss_values, eval_loss_values = [], []

for i in range(epoch):
    print("_"*100)
    print(f"epoch={i+1}")
    ## train ##
    model.train()
    total_loss = 0
    for step, batch in enumerate(train_batches):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        model.zero_grad()
        outputs = model(b_input_ids, token_type_ids=None,attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        loss.backward()
        total_loss += loss.item()
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_loss / len(train_batches)
    loss_values.append(avg_train_loss)
    print(f"train_loss={avg_train_loss}")
    
    ## test ##
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in eval_batches:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(eval_batches)
    eval_loss_values.append(eval_loss)
    print(f"test_loss={eval_loss}")

    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]

    print(f"Test Accuracy: {accuracy_score(pred_tags, valid_tags)*100} %")
    
    print("Test precision_score: {}".format(precision_score([pred_tags], [valid_tags])))
    print("Test recall_score: {}".format(recall_score([pred_tags], [valid_tags])))
    print("Test F1-Score: {}".format(f1_score([pred_tags], [valid_tags] , average='macro')))


____________________________________________________________________________________________________
epoch=1
train_loss=2.64115562489335
test_loss=0.7187817527188195
Test Accuracy: 59.08487014835669 %


/usr/local/lib/python3.8/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PAD seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


Test precision_score: 0.366592809663355
Test recall_score: 0.2117409074020559
Test F1-Score: 0.17088046415696248
____________________________________________________________________________________________________
epoch=2
train_loss=0.5830689861740864
test_loss=0.43197541766696507
Test Accuracy: 61.06823142378136 %
Test precision_score: 0.36923736687870773
Test recall_score: 0.2675436325029779
Test F1-Score: 0.23127670561587377
____________________________________________________________________________________________________
epoch=3
train_loss=0.42141887839411346
test_loss=0.32379403875933754
Test Accuracy: 67.00724385537595 %
Test precision_score: 0.42434422128511184
Test recall_score: 0.29534374689085663
Test F1-Score: 0.24430994876527684
____________________________________________________________________________________________________
epoch=4
train_loss=0.3580798754389857
test_loss=0.287511155837112
Test Accuracy: 69.25790023091766 %
Test precision_score: 0.4559359588306769
Test

# Save Model


<div dir = "rtl" style = 'font-family: "B Zar";'>
  <p>در نهایت مدل را میتوانیم با این دستورات سیو کنیم </p>

</div>


In [ ]:
filename = 'ner_bert_model.sav'
joblib.dump(model, filename)

['ner_bert_model.sav']

# Test Model


<div dir = "rtl" style = 'font-family: "B Zar";'>
  <p>سپس با گرفتن کوئری میتوان خروجی مدل آموزش دیدمان را بر روی این جمله مشاهده کنیم که به هر کلمه چه موجودیت نامداری اختصاص میدهد. که میبیینم مدل دقت خیلی بالایی ندارد به علت دشوار بودن تسک و همچنین محدودیت سخت افزاری برای ترین با زمان های زیاد بر روی دادگان که حجم بسیار بالایی نیز داشتند که البته در صورت سوال نیز خواسته به دقت بهینه رسیدن نیز نبود و مراحل پیاده سازی ذکر شده بود اولویت دارد

</p>
</div>


In [ ]:
query = "Professor gave justice to his role. What a legend. you'll start admiring some characters. Who thought a TV series on a money heist will be so interesting."

In [ ]:

tokenized_query = tokenizer.encode(query)
model_query = torch.tensor([tokenized_query]).cuda()
with torch.no_grad():
    output = model(model_query)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
tokens = tokenizer.convert_ids_to_tokens(model_query.to('cpu').numpy()[0])
new_tokens, new_tags = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        new_tags.append(tag_values[label_idx])
        new_tokens.append(token)

In [ ]:
for token, label in zip(new_tokens, new_tags):
    print(f"{label}\t{token}")

O	[CLS]
O	Professor
O	gave
B-REVIEW	justice
O	to
O	his
O	role
O	.
O	What
O	a
B-GENRE	legend
O	.
B-QUOTE	you
O	'
I-QUOTE	ll
O	start
I-OPINION	admiring
O	some
I-PLOT	characters
O	.
O	Who
O	thought
O	a
O	TV
I-GENRE	series
I-PLOT	on
I-PLOT	a
I-PLOT	money
I-PLOT	heist
O	will
O	be
O	so
B-GENRE	interesting
PAD	.
PAD	[SEP]
